In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Upperbound para os pontos
N = 300
#Número de hospitais
H = 20
#Número de veículos
K = 50
#Número de Ocorrências
O = 15

In [3]:
#Cria uma matriz de coordenadas (x,y) aleatórias
coord = pd.DataFrame(100*np.random.rand(N,2),index=range(N),columns=["x","y"])
#Verifica se existem dois pontos iguais
if max(coord.value_counts()) > 1:
    count = 0
    for i in coord.value_counts():
        if i>1:
            #e então remove todos os que se aplicarem
            coord.drop(count)
        count+=1
    #por fim reindexando a matriz
    coord.index=pd.RangeIndex(0,len(coord))

In [4]:
#Cria as colunas com os arcos do vértice
coord[1] = coord[2] = coord[3] = coord[4] =-1

In [5]:
#Matriz de distância entre os pontos
_DistanceMatrix = pd.DataFrame(0.0,index=range(len(coord)),columns=range(len(coord)))

In [6]:
#Popula a matriz de distância com
#a distância euclidiana
for i in _DistanceMatrix.index:
    for j in _DistanceMatrix.index:
        dx = (coord.loc[i]["x"]-coord.loc[j]["x"])**2
        dy = (coord.loc[i]["y"]-coord.loc[j]["y"])**2
        _DistanceMatrix.loc[i][j] = (dx+dy)**(1/2)

In [7]:
#Lista de todos os nodos com arcos completos
_ConnectionMatrix = pd.DataFrame(False,index=range(len(coord)),columns=range(len(coord)))
completed = []
d={x:0 for x in _ConnectionMatrix.index}
for i in _ConnectionMatrix.index:
    #Contabiliza o número de conexões já feitas no nodo atual
    n_con = sum(_ConnectionMatrix.loc[i])
    #Se ainda faltar alguma conexão livre
    if n_con < 4:
        #Procura todos os nodos que ainda não foram completados
        incomplete = ~_ConnectionMatrix.index.isin(completed)
        
        #E todos os nodos que ainda não foram preenchidos em 'i'
        free = ~_ConnectionMatrix.loc[i].values
        
        #Ordena em ordem crescente todas as distâncias que satisfazem
        #as duas condições e pega os melhores arcos para completar 'i'
        sort = sorted(_DistanceMatrix.loc[i][incomplete&free])[1:1+(4-n_con)]
        
        #Separa quais os índices dos pontos de chegada determinados
        #na ordenação anterior
        bol = _DistanceMatrix.loc[i].isin(sort)
        
        string = str(i)+": "+str(_ConnectionMatrix.index[~free].values)+" -> "
        #Percorre esses indices
        for j in _DistanceMatrix.loc[i][bol].index:
        
            #Cria o arco 'i_j'
            _ConnectionMatrix.at[i,j] = True
            
            #Cria o arco 'j_i'
            _ConnectionMatrix.at[j,i] = True
            d[j]+=1
            d[i]+=1
            #Se 'j' completar adicionar 'j' na lista de nodos completos
            if sum(_ConnectionMatrix.loc[j]) == 4:
                completed.append(j)
        
        #Se 'i' completar adicionar 'i' na lista de nodos completos
        if sum(_ConnectionMatrix.loc[i]) == 4:
            completed.append(i)
        free = ~_ConnectionMatrix.loc[i].values
        string+= str(_ConnectionMatrix.index[~free].values)
        print(string)

0: [] -> [ 11 192 253 296]
1: [] -> [ 89 220 227 287]
2: [] -> [178 215 230 236]
3: [] -> [ 68  77 124 266]
4: [] -> [ 30  53 261 279]
5: [] -> [107 145 161 255]
6: [] -> [ 14  45  99 274]
7: [] -> [ 20  76  82 297]
8: [] -> [ 24  43 172 237]
9: [] -> [ 18 168 170 233]
10: [] -> [ 68 149 183 245]
11: [0] -> [  0 189 192 296]
12: [] -> [116 165 184 196]
13: [] -> [ 87 158 275 277]
14: [6] -> [  6  45 111 274]
15: [] -> [ 74 151 173 252]
16: [] -> [ 89 224 227 287]
17: [] -> [ 37 138 195 240]
18: [9] -> [  9 170 233 298]
19: [] -> [ 26 200 202 278]
20: [7] -> [  7  76  82 226]
21: [] -> [ 64  66 232 289]
22: [] -> [ 38  84 194 220]
23: [] -> [ 59  97 134 275]
24: [8] -> [  8  43 237 249]
25: [] -> [ 54 137 162 185]
26: [19] -> [ 19 112 123 180]
27: [] -> [154 163 211 281]
28: [] -> [ 79 181 216 241]
29: [] -> [ 71  75  85 218]
30: [4] -> [  4  53 132 279]
31: [] -> [ 81 133 174 177]
32: [] -> [ 43 171 172 237]
33: [] -> [ 55 161 186 246]
34: [] -> [ 91 150 160 175]
35: [] -> [ 52  61  73

In [8]:
#Verifica se a instância é válida
for i in _ConnectionMatrix.index:
    if sum(_ConnectionMatrix.loc[i]) != 4:
        print(i)
        print(sum(_ConnectionMatrix.loc[i]))

In [9]:
#Preenche os arcos para cada ponto
for i in _ConnectionMatrix.index:
    #Pega todos os nodos em que existe um arco saindo de 'i'
    index = _ConnectionMatrix.index[_ConnectionMatrix.loc[i]]
    count=0
    
    for j in coord.columns[2:2+len(index)]: 
        coord.at[i,j] = index[count] #Adiciona 'ij' aos arcos de 'i'
        count+=1

In [10]:
sinal = 0 #Tempo entre as alterações de estado de um sinal
for i in coord.index:
    for j in coord.loc[i][2:]:
        #Somamos o tempo de cada arco
        if j != -1:
            sinal+=_DistanceMatrix.loc[i][j]
sinal = (sinal/(3*len(coord))) #O tempo de alteração vai ser a média

In [17]:
#Matriz de estado dos arcos entre os sinais


for i in coord.index:
    for n in coord.loc[i][2:]:
        if n != -1:
            j = int(n)
            #Se o tempo de percorrer o arco 'ij' é menor que o tempo
            #de alteração dos sinais SignalState = True, False caso contrário
            SignalState.loc[i][n] = (_DistanceMatrix.loc[i][j] < sinal)

In [36]:
SignalState = pd.DataFrame(np.random.random((N,4))<0.5,index = coord.index, columns= coord.columns[2:],dtype=bool)

In [42]:
#Matriz de Estado Instântaneo, representa o estado dos sinais no momento 0
SignalInstantState = pd.DataFrame(False,index = coord.index, columns=["state"], dtype=bool)
SignalInstantState.loc[0][0]=True


visited = set()
#Nodo inicial
actual_node = 0

#Busca por profundidade, para encontrar ciclo principal
def dfs(visited,actual_node):
    visited.add(actual_node)
    not_visited = coord.loc[actual_node][2:][~coord.loc[actual_node][2:].isin(visited)]
    if len(not_visited)>0:
        for i in not_visited:
            if i != -1:
                #Pega a coluna que representa o nodo de índice 'i'
                column = coord.loc[actual_node][coord.loc[actual_node] == int(i)].index[0]
                #Realiza um XOR
                xor = SignalState.loc[actual_node][column] != SignalInstantState.loc[actual_node]["state"]
                #Prevê o estado do nodo 'i' no instante 0
                SignalInstantState.loc[int(i)]["state"] = xor
                
                index = coord.loc[actual_node][2:][coord.loc[actual_node][2:]==i].index[0]
                #Adiciona o nodo 'i'
                visited = dfs(visited,i)
        return visited
            
    else:
        return visited
#Lista dos nodos no ciclo principal
l = dfs(visited,0)

In [45]:
#Todos os índices de ciclos externos
aux = coord.index[~coord.index.isin(l)]
for i in aux:
    #Eliminação de ciclos externos
    coord.drop(i,inplace=True)
len(coord)

300

In [46]:
#Eliminação dos Índices de ciclos externos
SignalInstantState.drop(SignalInstantState.loc[~SignalInstantState.index.isin(coord.index)].index,inplace=True)
SignalState.drop(SignalState.loc[~SignalState.index.isin(coord.index)].index,inplace=True)

#Reindexação Geral
if len(coord) < N:
    for i in range(len(coord)):
        for j in range(len(coord)):
            for k in coord.loc[coord.index[j]][2:].index:
                if coord.loc[coord.index[j]][k] == coord.index[i]:
                    coord.loc[coord.index[j],k] = i
    coord.index = pd.RangeIndex(len(coord))
    SignalInstantState.index = pd.RangeIndex(len(coord))
    SignalState.index = pd.RangeIndex(len(coord))

In [47]:
#Pontos escolhidos como hospitais
hospitais = pd.DataFrame(coord.index[np.random.randint(0,len(coord),H)],columns=["P"])

In [48]:
#Instância dos Veículos
veiculos = pd.DataFrame(index=range(K),columns=["Inicial","LowerBound","UpperBound"])

for i in veiculos.index:
    #Ponto inicial do veículo, escolhido aleatoriamente
    inicial = hospitais.loc[np.random.randint(0,len(hospitais))]["P"]
    #Quantidade do mesmo tipo de veículo no mesmo hospital
    qntd = np.random.randint(1,5)
    #Para cada veículo do mesmo tipo
    for j in range(qntd):
        lb = np.random.randint(1,6)      #Temos a prioridade mínima pela qual ele atende
        ub = np.random.randint(5,11)     #e a prioridade máxima que ele consegue atender
        veiculos.loc[i+j] = [inicial,lb,ub] #Inserção do veículo na instância

In [49]:
#Ocorrências
ocurrences = pd.DataFrame(index=range(O),columns=["P"])
#Pontos que não estejam previamente ocupados por hospitais
aux = coord[~coord.index.isin(hospitais["P"].values)].index
for i in range(O):
    #Escolhe aleatoriamente dentro dos pontos livres
    ocurrences.loc[i]["P"] = aux[np.random.randint(0,len(aux))]
    

In [50]:
#Instância das vítimas
victims = pd.DataFrame(index=range(O),columns=["Ponto","Prioridade"])
for i in range(O):
    #Escolhe dentro dos pontos das ocorrências 
    victims.loc[i] = [ocurrences.loc[i]["P"],np.random.randint(1,11)]

In [52]:
#Salva as instâncias
veiculos.to_csv("k_"+str(K))
victims.to_csv("v_"+str(O))
coord.to_csv("n_"+str(len(coord)))
SignalInstantState.to_csv("si_"+str(len(SignalInstantState)))
SignalState.to_csv("s_"+str(len(SignalInstantState)))
_DistanceMatrix.to_csv("d_"+str(len(_DistanceMatrix)))

#Gera uma imagem do sistema criado
plt.figure(figsize=(30,30))
for i in coord.index:
    plt.scatter(coord.loc[i]["x"],coord.loc[i]["y"],s=20,color='black',zorder=1)
    plt.text(coord.loc[i]["x"],coord.loc[i]["y"],str(i),fontsize=12)

visitados = {x:[] for x in coord.index}
for i in coord.index:
    l = [int(x) for x in coord.loc[i][2:][coord.loc[i][2:] != -1]]
    xcoord = list(coord.loc[l]['x'].values)
    
    ycoord = list(coord.loc[l]['y'].values)
    
    for j in range(len(xcoord)):
        if l[j] not in visitados[i]:
            if _DistanceMatrix.loc[i][l[j]] < sinal:
                plt.plot([coord.loc[i]["x"],xcoord[j]],[coord.loc[i]["y"],ycoord[j]],'g--',zorder=0)
            else:
                plt.plot([coord.loc[i]["x"],xcoord[j]],[coord.loc[i]["y"],ycoord[j]],"r--",zorder=0)
        visitados[l[j]].append(i)

plt.savefig("n"+str(len(coord)))